In [1]:
# This is used to increase the notebook's width to fill the screen, allowing for better plot visualization
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import os
import cv2
import time
import shutil
import numpy as np
import pandas as pd

from tqdm import tqdm

C:\Users\leoar\AppData\Local\Temp\ipykernel_7568\2832268761.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Path to Dataset

In [2]:
# Relative path to dataset
data_dir = os.path.join( "C:\\Datasets", "COVID19", "Tomografia", "COVIDx CT-3A" )
assert os.path.exists( data_dir ), "Unable to find the relative path to COVIDx CT-3A, please check data_dir..."

export_dir = os.path.join( "C:\\Datasets", "COVID19", "CT", "classification" )
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

# Path to metadata csv
csv_path = os.path.join( data_dir, "new_split_metadata.csv" )

# Reads metadata as dataframe, "age" column is treated as str since "N/A" can't be int
samples_df = pd.read_csv(csv_path, sep = ";", na_filter = False, dtype={"age": str})
print( len(samples_df) )
samples_df.head()

425024


,filename,patient_id,source,class,country,sex,age,partition,slice_selection,x_min,y_min,x_max,y_max,verified_finding,view,modality
0,NCP_96_1328_0032.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,9,94,512,405,Yes,Axial,CT
1,NCP_96_1328_0035.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,10,106,512,405,Yes,Axial,CT
2,NCP_96_1328_0036.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,10,105,512,406,Yes,Axial,CT
3,NCP_96_1328_0037.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,11,104,512,406,Yes,Axial,CT
4,NCP_96_1328_0038.png,NCP_96,CNCB,COVID-19,China,M,74.0,train,Expert,11,103,512,406,Yes,Axial,CT


# Resize Images

In [3]:
def save_img( exp_path, src_img ):
    # Saves the resized image (creates saving directory if needed)
    exp_dir = os.path.dirname(exp_path)
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    cv2.imwrite(exp_path, src_img)
    return

def copy_file( src_path, dst_path ):
    # Copies a file
    dst_dir = os.path.dirname(dst_path)
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    shutil.copy2(src_path, dst_path)
    return

def resize_samples( df, import_dir, sub_dir, export_dir, dataset, sizes, seed = 42 ):
    
    # Sets a seed for reprodutibility
    np.random.seed( seed )
    
    # Filters inputed df to select only rows from the current source
    source_df = df[ df["source"] == dataset ].copy(deep = True)
    source_df.reset_index(drop = True, inplace = True)
    
    # Defines the interpolation method used for each sample. Used Interpolations are: 
    # 0 = INTER_NEAREST, 1 = INTER_LINEAR, 2 = INTER_CUBIC, 3 = INTER_AREA, 4 = INTER_LANCZOS4
    # The same image will be resized with the same interpolation regardless of the output shape
    source_df["interpolation"] = source_df.apply(lambda row: np.random.randint(0, 5), axis = 1)
    
    # Converts to dict for faster iteration through rows
    df_dict = source_df.to_dict( "records" )
    for row in tqdm(df_dict):
        
        # Paths to import the original image and export the resized ones
        import_path = os.path.join( import_dir, sub_dir, row["filename"] )
        ex_csv_path = os.path.join( row["source"], row["partition"], row["filename"] )
        
        # Lists all resized paths for the current file
        export_paths = []
        for size in sizes:
            if size == "original":
                export_paths.append( os.path.join( export_dir, "original", ex_csv_path ) )
            else:
                export_paths.append( os.path.join( export_dir, f"{size}x{size}", ex_csv_path ) )
                
        # Skips current file if all resized images exist
        if all([os.path.exists(p) for p in export_paths]):
            continue
        
        # Loads original image
        src_img = cv2.imread( import_path, -1 )
        
        for size, export_path in zip(sizes, export_paths):
            if os.path.exists(export_path):
                continue
                
            if size == "original":
                copy_file( import_path, export_path )
            
            else:
                # Resizes using a randomly chosen interpolation method and saves the resulting image
                dst_img = cv2.resize( src_img, (size, size), interpolation = row["interpolation"] )
                save_img( export_path, dst_img )
    
    # Saving dataset with updated paths
    source_df["path"] = source_df.apply( lambda row: os.path.join(row["source"], row["partition"], row["filename"]), axis = 1)
    
    # Creates a folder for the CSV files if needed
    csv_dir = os.path.join("..", "metadata")
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir)
    
    # Saves the dataframe
    dataset_csv_path = os.path.join( csv_dir, "{}_data.csv".format(dataset) )
    source_df.to_csv( dataset_csv_path, index = False, sep = ";" )
    
    return

In [4]:
# List of unique sources (datasets used to build COVIDx CT-3A)
source_list = [ "radiopaedia.org", "LIDC-IDRI", "COVID-CTset", "Stony Brook", 
                "COVID-CT-MD", "iCTCF", "CNCB", "COVID-19-CT-Seg", "TCIA", "STOIC" ]

for src in np.unique( samples_df["source"].to_list() ):
    print(src in source_list, src)
print("\n")

# sizes_list = [224, 240, 260, 299, 300, "original"]
sizes_list = [240, 260]

for idx, src in enumerate(source_list):
    print( "{}/{} - {}:\n".format(idx+1, len(source_list), src) )
    resize_samples( samples_df, data_dir, "3A_images", export_dir, src, sizes_list )
    print("\n")

True CNCB
True COVID-19-CT-Seg
True COVID-CT-MD
True COVID-CTset
True LIDC-IDRI
True STOIC
True Stony Brook
True TCIA
True iCTCF
True radiopaedia.org


1/10 - radiopaedia.org:



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3574/3574 [01:37<00:00, 36.61it/s]




2/10 - LIDC-IDRI:



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3999/3999 [01:36<00:00, 41.48it/s]




3/10 - COVID-CTset:



...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1726/1726 [00:34<00:00, 49.84it/s]




9/10 - TCIA:



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11816/11816 [03:53<00:00, 50.62it/s]




10/10 - STOIC:



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192361/192361 [1:07:49<00:00, 47.27it/s]


# Verify Paths

In [5]:
def check_paths( i_dir, data_source, sizes ):
    
    metadata_csv_path = os.path.join(i_dir, "{}_data.csv".format(data_source))
    df = pd.read_csv(metadata_csv_path, sep = ";", na_filter = False, dtype={"age": str})
    
    # List of all paths
    path_list = df["path"].to_list()

    # List of paths per partition
    train_path_list = df[df["partition"] == "train"]["path"].to_list()
    val_path_list   = df[df["partition"] ==   "val"]["path"].to_list()
    test_path_list  = df[df["partition"] ==  "test"]["path"].to_list()

    for p in tqdm(path_list):
    
        for size in sizes:

            path = os.path.join( i_dir, f"{size}x{size}", p )

            if not os.path.exists( path ):
                print("\n\tPath '{}' does not exist...".format(path))

            num_partitions = np.sum([ (p in train_path_list), (p in val_path_list), (p in test_path_list) ])

            if num_partitions > 1:
                print("\n\tSample '{}' belongs to >1 partition...".format(path))

            if num_partitions < 1:
                print("\n\tSample '{}' does not belong to any partition...".format(path))
    
    return

In [6]:
dataset_list = list(source_list)

# Relative path to dataset
assert os.path.exists( export_dir ), "Unable to find the relative path to resized images, please check image_dir..."

for idx, dataset_name in enumerate(dataset_list):
    print( "{}/{} - {}:\n".format(idx+1, len(dataset_list), dataset_name) )
    check_paths( export_dir, dataset_name, sizes_list )
    print("\n\n")
    

1/10 - radiopaedia.org:



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3574/3574 [00:01<00:00, 2137.23it/s]





2/10 - LIDC-IDRI:



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Datasets\\COVID19\\CT\\classification\\LIDC-IDRI_data.csv'